In [1]:
import pandas , numpy
import psycopg2
from dotenv import load_dotenv
import os
import time

In [2]:
load_dotenv()

True

In [3]:
#CONNECTING TO COMPANY DB
db_name = "OpenAlex"
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")

In [4]:
conn = psycopg2.connect(f"dbname ={db_name} user={user} password={password} host={host} port={port}")

In [5]:
#getting top 5 papers for each field
#bronchitis
query = """
        WITH cardiovascular_related_works AS (
            SELECT DISTINCT id
            FROM works
            JOIN works_mesh wm ON works.id = wm.work_id
            WHERE wm.descriptor_name ILIKE ANY(ARRAY['%lymphoma%', '%Hodgkin%', '%lymphatic%'])
        ),
        work_institutions AS (
            SELECT
                crw.id AS work_id,
                unnest(w.corresponding_institution_ids) AS institution_id
            FROM cardiovascular_related_works crw
            JOIN works w ON crw.id = w.id
            WHERE w.corresponding_institution_ids IS NOT NULL
        )
        SELECT
            i.id AS institution_id,
            i.display_name AS institution_name,
            COUNT(DISTINCT wi.work_id) AS num_publications,
            COALESCE(SUM(w.cited_by_count), 0) AS total_citations
        FROM work_institutions wi
        JOIN institutions i ON wi.institution_id = i.id
        JOIN works w ON wi.work_id = w.id
        GROUP BY
            i.id,
            i.display_name
        ORDER BY
            total_citations DESC
        LIMIT 10;
        """

In [ ]:
cursor = conn.cursor()
cursor.execute(query)
#cursor.execute(query , (["%cancer%"],))
rows = cursor.fetchall()
print(rows)

In [7]:
dict_diseases = {'Bronchitis': ['%bronchitis%'],
 'Covid': ['%covid%', '%coronavirus%', '%sars-cov-2%', '%2019-ncov%'],
 "Alzheimer's": ['%alzheimer%', '%dementia%', '%neurodegenerative%'],
 'Diabetes': ['%diabetes%', '%hyperglycemia%'],
 'Lymphoma': ['%lymphoma%', '%Hodgkin%', '%lymphatic%'],
 'Sarcoma': ['%sarcoma%'],
 'Liver Disease': ['%liver%',
  '%hepatic%',
  '%cirrhosis%',
  '%hepatitis%',
  '%cholestasis%'] ,
  'Heart disease': ['%heart%',
  '%cardiovascular%',
  '%cardiac%',
  '%coronary%',
  '%myocardial%',
  '%angina%',
  '%cardiomyopathy%',
  '%arteriosclerosis%'],
  'Cancer': ['%cancer%', '%carcinoma%', '%neoplasm%', '%tumor%', '%malignant%']}

In [8]:
def running_top_institutions(keyword = None, lst_keywords = None):
    if keyword:
        query = f"""
        WITH cardiovascular_related_works AS (
            SELECT DISTINCT id
            FROM works
            JOIN works_mesh wm ON works.id = wm.work_id
            WHERE wm.descriptor_name ILIKE '{keyword}'
        ),
        work_institutions AS (
            SELECT
                crw.id AS work_id,
                unnest(w.corresponding_institution_ids) AS institution_id
            FROM cardiovascular_related_works crw
            JOIN works w ON crw.id = w.id
            WHERE w.corresponding_institution_ids IS NOT NULL
        )
        SELECT
            i.id AS institution_id,
            i.display_name AS institution_name,
            COUNT(DISTINCT wi.work_id) AS num_publications,
            COALESCE(SUM(w.cited_by_count), 0) AS total_citations
        FROM work_institutions wi
        JOIN institutions i ON wi.institution_id = i.id
        JOIN works w ON wi.work_id = w.id
        GROUP BY
            i.id,
            i.display_name
        ORDER BY
            total_citations DESC
        LIMIT 10;
            """
    else:
        formatted_keywords = ", ".join([f"'{kw}'" for kw in lst_keywords])
        query = f"""
            WITH cardiovascular_related_works AS (
            SELECT DISTINCT id
            FROM works
            JOIN works_mesh wm ON works.id = wm.work_id
            WHERE wm.descriptor_name ILIKE ANY(ARRAY[{formatted_keywords}])
        ),
        work_institutions AS (
            SELECT
                crw.id AS work_id,
                unnest(w.corresponding_institution_ids) AS institution_id
            FROM cardiovascular_related_works crw
            JOIN works w ON crw.id = w.id
            WHERE w.corresponding_institution_ids IS NOT NULL
        )
        SELECT
            i.id AS institution_id,
            i.display_name AS institution_name,
            COUNT(DISTINCT wi.work_id) AS num_publications,
            COALESCE(SUM(w.cited_by_count), 0) AS total_citations
        FROM work_institutions wi
        JOIN institutions i ON wi.institution_id = i.id
        JOIN works w ON wi.work_id = w.id
        GROUP BY
            i.id,
            i.display_name
        ORDER BY
            total_citations DESC
        LIMIT 10;
            """
    return query

In [ ]:
#running all the diseases
for key , value in dict_diseases.items():
    print(f"running {key}")
    before_running = time.time()
    if len(value) == 1:
        query = running_top_institutions(keyword = value[0])
    else:
        query = running_top_institutions(lst_keywords = value)

    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    after_running = time.time()
    print(rows)
    print(f"Ran in {after_running - before_running} seconds")
    

"""
running Bronchitis
[('https://openalex.org/I2801388914', 'Southampton General Hospital', 9, Decimal('3565')), ('https://openalex.org/I43439940', 'University of Southampton', 9, Decimal('3336')), ('https://openalex.org/I2802542264', 'University Hospital of Basel', 1, Decimal('2182')), ('https://openalex.org/I900890020', 'University of Palermo', 4, Decimal('2054')), ('https://openalex.org/I4210087413', 'University of Colorado Health', 5, Decimal('1952')), ('https://openalex.org/I141945490', 'University of British Columbia', 14, Decimal('1930')), ('https://openalex.org/I4210096640', 'Royal Brompton Hospital', 34, Decimal('1891')), ('https://openalex.org/I4210138501', 'Chinese Academy of Agricultural Sciences', 34, Decimal('1631')), ('https://openalex.org/I47508984', 'Imperial College London', 12, Decimal('1503')), ('https://openalex.org/I4210099160', 'Centro de Investigación Biomédica en Red de Enfermedades Respiratorias', 15, Decimal('1355'))]
Ran in 194.40675592422485 seconds
running Covid
[('https://openalex.org/I154526488', 'Inserm', 1124, Decimal('245577')), ('https://openalex.org/I200296433', 'Chinese Academy of Medical Sciences & Peking Union Medical College', 770, Decimal('200655')), ('https://openalex.org/I204730241', 'Université Paris Cité', 495, Decimal('188728')), ('https://openalex.org/I83399316', 'Rockefeller University', 74, Decimal('184924')), ('https://openalex.org/I4210165077', 'Institut des Maladies Génétiques Imagine', 27, Decimal('160511')), ('https://openalex.org/I40120149', 'University of Oxford', 1051, Decimal('154811')), ('https://openalex.org/I37461747', 'Wuhan University', 581, Decimal('154139')), ('https://openalex.org/I136199984', 'Harvard University', 2042, Decimal('145970')), ('https://openalex.org/I47720641', 'Huazhong University of Science and Technology', 797, Decimal('141975')), ('https://openalex.org/I99065089', 'Tsinghua University', 305, Decimal('129789'))]
Ran in 388.70435786247253 seconds
running Alzheimer's
[('https://openalex.org/I136199984', 'Harvard University', 1475, Decimal('185605')), ('https://openalex.org/I45129253', 'University College London', 1487, Decimal('128064')), ('https://openalex.org/I154526488', 'Inserm', 1157, Decimal('113483')), ('https://openalex.org/I145311948', 'Johns Hopkins University', 925, Decimal('100711')), ('https://openalex.org/I180670191', 'University of California, San Francisco', 939, Decimal('98580')), ('https://openalex.org/I204465549', 'Washington University in St. Louis', 654, Decimal('97035')), ('https://openalex.org/I4210087915', 'Massachusetts General Hospital', 793, Decimal('96893')), ('https://openalex.org/I79576946', 'University of Pennsylvania', 752, Decimal('93866')), ('https://openalex.org/I36258959', 'University of California, San Diego', 743, Decimal('85549')), ('https://openalex.org/I1283280774', "Brigham and Women's Hospital", 507, Decimal('83725'))]
Ran in 457.1918411254883 seconds
running Sarcoma
[('https://openalex.org/I136199984', 'Harvard University', 624, Decimal('53636')), ('https://openalex.org/I1334819555', 'Memorial Sloan Kettering Cancer Center', 583, Decimal('48435')), ('https://openalex.org/I1343551460', 'The University of Texas MD Anderson Cancer Center', 573, Decimal('36573')), ('https://openalex.org/I4210140884', 'National Cancer Institute', 384, Decimal('33877')), ('https://openalex.org/I1283280774', "Brigham and Women's Hospital", 263, Decimal('29756')), ('https://openalex.org/I1299303238', 'National Institutes of Health', 231, Decimal('25117')), ('https://openalex.org/I4210087915', 'Massachusetts General Hospital', 360, Decimal('24681')), ('https://openalex.org/I154526488', 'Inserm', 338, Decimal('20788')), ('https://openalex.org/I1330342723', 'Mayo Clinic', 249, Decimal('15912')), ('https://openalex.org/I145311948', 'Johns Hopkins University', 219, Decimal('15138'))]
Ran in 167.66896557807922 seconds
"""

running Bronchitis
